In [5]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, OWL
import pandas as pd
import openpyxl

### Helper functions

In [6]:
def read_xls(input_file, left_namespace, right_namespace):

    # Load the workbook and select a worksheet
    wb = openpyxl.load_workbook(input_file, data_only=True)
    sheet = wb.active  # or wb['SheetName']

    # Read data from specific columns into lists
    left = []
    mapping = []
    right = []
    for row in sheet.iter_rows(min_row=2):
        left.append(row[0].value[len(left_namespace):])
        mapping.append(row[1].value)
        right.append(row[5].value[len(right_namespace):])

    # Create a pandas DataFrame from selected columns
    df = pd.DataFrame({
        'left': left,
        'mapping': mapping,
        'right': right
    })

    return(df)

In [7]:
def create_mapping(direction, mapping_file, left_prefix, left_namespace, right_prefix, right_namespace):
    # Create a new RDF graph
    g = Graph()

    # Add namespaces
    LEFT = Namespace(left_namespace)
    RIGHT = Namespace(right_namespace)
    g.bind(left_prefix, LEFT)
    g.bind(right_prefix, RIGHT)

    # Read the mapping file
    mapping_df = read_xls(mapping_file, left_namespace, right_namespace)

    # Add mappings to the graph
    for row in mapping_df.itertuples():
        left_term = row[1]
        mapping_term = row[2]
        right_term = row[3]
        left_uri = URIRef(LEFT[left_term])
        right_uri = URIRef(RIGHT[right_term])
        if mapping_term == 'e':
            g.add((left_uri, OWL.equivalentClass, right_uri))
        elif mapping_term == "ls" and direction == "ls" :
            g.add((left_uri, RDFS.subClassOf, right_uri))
        elif mapping_term == "rs" and direction == "rs" :
            g.add((right_uri, RDFS.subClassOf, left_uri))
        else:
            pass #no mapping

    return(g)

### Common core time to W3C time

In [1]:
mapping_file = "../matching/ccot-time-structural-matches.xlsx"

# Define a CCO namespace
left_prefix = "cco"
left_namespace = "https://www.commoncoreontologies.org/"

# Define a Time namespace
right_prefix = "to"
right_namespace = "http://www.w3.org/2006/time#"

# Map left to right
g = create_mapping(
    "ls",
    mapping_file,
    left_prefix,
    left_namespace,
    right_prefix,
    right_namespace
)

with open("../src/data/ccot-mapping.ttl", "w") as output_file:
    output_file.write(g.serialize(format="turtle"))

# Map right to left
g = create_mapping(
    "rs",
    mapping_file,
    left_prefix,
    left_namespace,
    right_prefix,
    right_namespace
)

with open("../src/data/time-mapping.ttl", "w") as output_file:
    output_file.write(g.serialize(format="turtle"))

NameError: name 'create_mapping' is not defined

### Common Core Units of Measure to QUDTS

In [15]:
mapping_file = "/home/sanne/Projects/Ontology-Tradecraft/projects/project-3/assignment/matching/bfo-core-ies-structural-matches.xlsx"

# Define a CCO namespace
left_prefix = "ies4"
left_namespace = "http://ies.data.gov.uk/ontology/ies4#"

# Define a Time namespace
right_prefix = "bfo"
right_namespace = "http://purl.obolibrary.org/obo/bfo#"

g = create_mapping(
    "rs",
    mapping_file,
    left_prefix,
    left_namespace,
    right_prefix,
    right_namespace
)

# Serialize the ontology to Turtle format and save
with open("../src/data/ies-mapping.ttl", "w") as output_file:
    output_file.write(g.serialize(format="turtle"))

g = create_mapping(
    "ls",
    mapping_file,
    left_prefix,
    left_namespace,
    right_prefix,
    right_namespace
)

with open("../src/data/bfo-mapping.ttl", "w") as output_file:
    output_file.write(g.serialize(format="turtle"))

In [9]:
! pwd

/home/sanne/Projects/Ontology-Tradecraft/projects/project-3/assignment/notebooks


In [ ]:
mapping_file = "../matching/ccom-qudt-structural-matches.xlsx"

# Define a CCO namespace
left_prefix = "cco"
left_namespace = "https://www.commoncoreontologies.org/"

# Define a Time namespace
right_prefix = "qudt-v1"
right_namespace = "http://qudt.org/schema/qudt#"

g = create_mapping(
    "ls",
    mapping_file,
    left_prefix,
    left_namespace,
    right_prefix,
    right_namespace
)

# Serialize the ontology to Turtle format and save
with open("../data/ccom-mapping.ttl", "w") as output_file:
    output_file.write(g.serialize(format="turtle"))

g = create_mapping(
    "rs",
    mapping_file,
    left_prefix,
    left_namespace,
    right_prefix,
    right_namespace
)

with open("../data/qudt-mapping.ttl", "w") as output_file:
    output_file.write(g.serialize(format="turtle"))